Please see: https://docs.google.com/document/d/1PKfTyH9sRCepNbbwP7isleTUXi1DSvZLqNZXbkfw8zQ/edit?tab=t.0

In these notes, we consider a simple mixed integer linear program (MILP) and derive a Benders decomposition scheme to solve it.

In [ ]:
# Ensure these packages are installed; if not, use the Pkg package and Pkg.add() function to install
using JuMP
using HiGHS

In [ ]:
f = [1, 4];
c = [2, 3];
e = [-2; -3];
A = [1 -3; -1 -3];
E = [1 -2; -1 -1];

The full monolithic model is as follows:

In [ ]:
monolithic_model = Model(HiGHS.Optimizer);
@variable(monolithic_model, x[1:2] >= 0, Int);
@variable(monolithic_model, y[1:2] >= 0);
@constraint(monolithic_model, A*x + E*y .<= e);
@objective(monolithic_model,Min, f'*x + c'*y);
latex_formulation(monolithic_model)

In [ ]:
optimize!(monolithic_model)

In [ ]:
value.(monolithic_model[:x])

Because we are considering a small example problem, the solver was able to quickly compute a solution. However, real-life mixed integer linear programs involve tens of thousands of integer decisions, pushing even the best commercial solvers to their computational limits. 

Note that if we fix the values of integer variables $x_1$ and $x_2$, we obtain a continuous linear program which can be easily solved. For this reason, Benders decomposition is often implemented to solve MILPs, considering integer decisions as complicating variables. 

Let's initialize the master (or planning) problem:

In [ ]:
master = Model(HiGHS.Optimizer);
@variable(master, x[1:2] >= 0, Int);
@variable(master,θ>=-1000) #### Initial lower bound on the operational cost (if operational cost is always positive, this can be set to 0)
@objective(master,Min, f'*x + θ);

latex_formulation(master)

Then, the subproblem is defined including auxiliary variables $z$ that act as local copies of the master variables:

In [ ]:
subprob = Model(HiGHS.Optimizer);
@variable(subprob, z[1:2] >= 0);
@variable(subprob, y[1:2] >= 0);
@constraint(subprob,A*z + E*y .<= e);
@objective(subprob, Min, c'*y);
latex_formulation(subprob)

The Benders decomposition algorithm, starts from solving the master problem:

In [ ]:
optimize!(master)

Using the solution of the master, we define a lower bound to the optimal value of our original MILP:

In [ ]:
LB = objective_value(master)

and also guesses for the master variables:

In [ ]:
xk = value.(master[:x])

We fix these guesses in the subproblem adding the constraints:

In [ ]:
fix.(subprob[:z],xk;force=true)
FixRef.(subprob[:z])

And solve it:

In [ ]:
optimize!(subprob)

We can now compute an upper bound to the optimal value of the original MILP as: 

In [ ]:
UB = f'*xk + objective_value(subprob)

With lower and upper bounds, we can compute the optimality gap, which gives a conserative estimate of the degree of sub-optimality of our current best guess:

In [ ]:
gap = (UB-LB)/abs(UB)

To improve our guesses, we need to include additional information into the master problem. 

Hence, we derive the so called optimality cuts, which are based on the dual solutions associated with the constraints fixing the values of the master variables in the subproblems:

In [ ]:
λ = dual.(FixRef.(subprob[:z]))

Having only one subproblem, we add one optimality cut per iteration to the master problem, which is given by:

$$\theta \geq f_k + \lambda^T(x - x_k)$$

where $f_k$ is the objective value of the subproblem obtained fixing variables $z$ to the value $x_k$.

Update the master problem by implementing the optimality cut above:

Then, repeat the above steps until you reach a zero gap. 

At each iteration: 

- Solve the upadted master problem
- Fix the values of the master variables in the subproblem
- Solve the subproblem to obtain an upper bound and compute the gap 
- If it is zero, then you are done! 
- If not zero, compute new optimality gaps and add them to the master problem, then repeat.

Then submit the optimal value you get and the final solution for vector $x$. Report also on how many iterations it took, and write out the cuts computed along the way.

**Bonus**: try to visualize the progression of the Benders algorithm by making a 3d plot of the cuts in $(\theta, x_1, x_2)$ coordinates. The total objective function value should appear as a surface, while the optimality cuts should be planes. Then illustrate the guesses for $x$ for each iteration by showing points $(\theta, x_1, x_2)$ on the surface.